In [87]:
from pyspark.sql import SparkSession
import pandas as pd
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("GX_1") \
    .getOrCreate()
raw_df = spark \
    .read \
    .option("header","true") \
    .csv("file:///home/padmapriya/GX/Kickstarter.csv")

In [2]:
raw_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- currency: string (nullable = true)
 |-- main_category: string (nullable = true)
 |-- sub_category: string (nullable = true)
 |-- launched_at: string (nullable = true)
 |-- deadline: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- goal_usd: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- country: string (nullable = true)
 |-- blurb_length: string (nullable = true)
 |-- name_length: string (nullable = true)
 |-- status: string (nullable = true)
 |-- start_month: string (nullable = true)
 |-- end_month: string (nullable = true)
 |-- start_Q: string (nullable = true)
 |-- end_Q: string (nullable = true)
 |-- usd_pledged: string (nullable = true)



In [3]:
raw_df.createOrReplaceTempView("campaigns")

In [4]:
type(raw_df)

pyspark.sql.dataframe.DataFrame

In [5]:
raw_df.show(20,truncate=False)

+----------+---------------------------------------------------------------+--------+-------------+--------------+-------------------+-------------------+--------+------------------+-------------+-------+-------+------------+-----------+----------+-----------+---------+-------+-----+-----------------+
|id        |name                                                           |currency|main_category|sub_category  |launched_at        |deadline           |duration|goal_usd          |city         |state  |country|blurb_length|name_length|status    |start_month|end_month|start_Q|end_Q|usd_pledged      |
+----------+---------------------------------------------------------------+--------+-------------+--------------+-------------------+-------------------+--------+------------------+-------------+-------+-------+------------+-----------+----------+-----------+---------+-------+-----+-----------------+
|1687733153|Socks of Speed and Socks of Elvenkind                          |USD     |games 

In [6]:
raw_df.toPandas()

,id,name,currency,main_category,sub_category,launched_at,deadline,duration,goal_usd,city,state,country,blurb_length,name_length,status,start_month,end_month,start_Q,end_Q,usd_pledged
0,1687733153,Socks of Speed and Socks of Elvenkind,USD,games,Tabletop Games,2018-10-30 20:00:02,2018-11-15 17:59:00,16.0,2000.0,Menasha,WI,US,14,7,successful,10,11,Q4,Q4,6061.0
1,227936657,Power Punch Boot Camp: An All-Ages Graphic Novel,GBP,comics,Comic Books,2018-08-06 10:00:43,2018-09-05 10:00:43,30.0,3870.99771,Shepperton,England,GB,24,8,successful,8,9,Q3,Q3,3914.505120400001
2,454186436,"""Live Printing with SX8: """"Squeegee Pulp Up""""""",USD,fashion,Apparel,2017-06-09 15:41:03,2017-07-09 15:41:03,30.0,1100.0,Manhattan,NY,US,21,7,successful,6,7,Q2,Q3,1110.0
3,629469071,Lost Dog Street Band's Next Album,USD,music,Country & Folk,2014-09-25 18:46:01,2014-11-10 06:00:00,45.0,3500.0,Nashville,TN,US,15,6,successful,9,11,Q3,Q4,4807.0
4,183973060,"Qto-X, a Tiny Lantern",USD,technology,Gadgets,2016-11-28 16:35:11,2017-01-27 16:35:11,60.0,30000.0,Troy,MI,US,15,4,successful,11,1,Q4,Q1,40368.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192543,1882418986,Book CafÃÂ© Bergen,NOK,food,Drinks,2016-10-19 18:07:51,2016-12-18 19:07:51,60.0,57858.66499999999,Bergen,Hordaland Fylke,NO,21,3,failed,10,12,Q4,Q4,1.2242605
192544,89212319,Chainlink tool,NOK,technology,Fabrication Tools,2014-11-14 12:11:46,2014-12-31 12:59:00,47.0,115717.33,BÃÂ¥tsfjord,Finnmark Fylke,NO,18,2,failed,11,12,Q4,Q4,157.27393930000005
192545,109884142,PNW Juice Truck,USD,food,Drinks,2016-02-02 22:34:22,2016-03-03 22:34:22,30.0,30000.0,Seattle,WA,US,13,3,failed,2,3,Q1,Q1,0.0
192546,277520923,succulent christmas tree,USD,art,Conceptual Art,2014-08-06 18:16:29,2014-10-05 18:16:29,60.0,1200.0,Los Angeles,CA,US,3,3,failed,8,10,Q3,Q4,143.0


In [7]:
from great_expectations.dataset import SparkDFDataset

In [9]:
raw_test_df=SparkDFDataset(raw_df)
type(raw_test_df)

2023-03-18 19:01:24,980 WARN execution.CacheManager: Asked to cache already cached data.


great_expectations.dataset.sparkdf_dataset.SparkDFDataset

## 1.Check Mandatory columns

In [11]:
mandatory_columns=["id","currency","main_currency","launched_at","deadline","country","status","usd_pledged"]

In [13]:
for column in mandatory_columns:
    try:
        assert raw_test_df.expect_column_to_exist(column).success,f"column {column} does not exist:FAILED"
        print(f"column {column} exists:PASSED")
    except AssertionError as e:
        print(e)

column id exists:PASSED
column currency exists:PASSED
column main_currency does not exist:FAILED
column launched_at exists:PASSED
column deadline exists:PASSED
column country exists:PASSED
column status exists:PASSED
column usd_pledged exists:PASSED


## 2.Mandatory columns should not be null

In [18]:
from pyspark.sql.utils import AnalysisException
mandatory_cols=["id","currency","main_category","launched_at","deadline","country","status","usd_pledged"]
for column in mandatory_cols:
    try:
        test_result = raw_test_df.expect_column_values_to_not_be_null(column)
        assert test_result.success, f"{test_result.result['unexpected_count']} of {test_result.result['element_count']} items in column {column} are null:FAILED"
        print(f"All items in column {column} are not null:PASSED")
    except AssertionError as e:
        print(e)
    except AnalysisException as e:
        print(e)

All items in column id are not null:PASSED
All items in column currency are not null:PASSED
1 of 192548 items in column main_category are null:FAILED
1 of 192548 items in column launched_at are null:FAILED
1 of 192548 items in column deadline are null:FAILED
1 of 192548 items in column country are null:FAILED
1 of 192548 items in column status are null:FAILED
1 of 192548 items in column usd_pledged are null:FAILED


## 3.Check for valid date format

In [24]:
#test_result=raw_test_df.expect_column_values_to_match_strftime_format('launched_at','%Y-%m-%d %H:%M:%S')
test_result=raw_test_df.expect_column_values_to_match_regex('launched_at',r"^\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}$")
f""""{round(test_result.result['unexpected_percent'],2)}% is not a valid date time format"""

'"0.29% is not a valid date time format'

In [25]:
test_result=raw_test_df.expect_column_values_to_match_regex('deadline',r"^\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}$")
f""""{round(test_result.result['unexpected_percent'],2)}% is not a valid date time format"""

'"0.04% is not a valid date time format'

## 4.Check for uniqueness

In [27]:
test_result=raw_test_df.expect_column_values_to_be_unique("id")
failed_msg = f"{test_result.result['unexpected_count']} of {test_result.result['element_count']} items or {round(test_result.result['unexpected_percent'],2)}% are not unique:FAILED"
print(f"column id is unique:PASSED" if test_result.success else failed_msg)

48162 of 192548 items or 25.01% are not unique:FAILED


# Filter Data

### Business Rules:
1.filter campaigns in US only
2.filter campaigns that are active in 2017 and 2018 based on "launch_at" and "deadline"
3.include only a list of categories
4.include successful campaigns only
5.include USD currency

In [28]:
main_categories=["art","publishing","film & video","technology","journalism","food","dance","photography","games","crafts","music","comics","theater","design"]
assessment_year=['2017','2018']
country='US'
currency='USD'

In [39]:
# reference data for assessment year
# assessment_yr_refer=[("2017","2016-07-01","2017-06-30"),("2018","2017-07-01","2018-06-30")]
# cols=['assessment_year','period_start_dt','period_end_dt']
# ay_df=spark.createDataFrame(assessment_yr_refer,cols)
# ay_df_ref=spark.createOrReplaceView("ay_ref")

In [40]:
ay_df=spark.read.option("header","true").csv("file:///home/padmapriya/GX/ay_data.csv")

In [41]:
ay_df.show()

+---------------+---------------+-------------+
|assessment_year|period_start_dt|period_end_dt|
+---------------+---------------+-------------+
|           2017|     2016-07-01|   2017-06-30|
|           2018|     2017-07-01|   2018-06-30|
+---------------+---------------+-------------+



In [44]:
ay_df_ref=ay_df.createOrReplaceTempView("ay_ref")

## Apply transformation

In [47]:
filtered_df = spark.sql(f"""
    SELECT id,
           name,
           currency,
           main_category,
           launched_at,
           deadline,
           goal_usd,
           country,
           usd_pledged,
           status,
           assessment_year
    FROM (SELECT t.*,
               ay.assessment_year,
               row_number() OVER (
                   PARTITION BY t.id
                   ORDER BY t.launched_at, 
                            ay.assessment_year DESC) row_no
          FROM campaigns t
          INNER JOIN ay_ref ay
              ON TO_DATE(t.launched_at) <= ay.period_end_dt 
              AND t.deadline > ay.period_start_dt
          WHERE country = '{country}'
          AND status = 'successful'
          AND main_category IN ('{"','".join(main_categories)}')
          AND ay.assessment_year IN ('{"','".join(assessment_year)}')
          AND currency = '{currency}'
   ) WHERE row_no = 1 
    """)
filtered_df.createOrReplaceTempView("filtered_campaigns")
filtered_df.toPandas()

,id,name,currency,main_category,launched_at,deadline,goal_usd,country,usd_pledged,status,assessment_year
0,10064783,WarmUp: A High-Protein Coffee,USD,food,2016-11-14 17:55:34,2016-12-14 17:55:34,8000.0,US,8599.0,successful,2017
1,1010419089,Silver Linings by J.R. Mounts,USD,comics,2018-01-10 02:10:01,2018-02-16 05:00:00,500.0,US,2914.5,successful,2018
2,1017132344,Princes of the Universe: a Nino Malong Art Book,USD,publishing,2016-07-31 19:03:04,2016-09-05 01:00:00,1200.0,US,1670.0,successful,2017
3,1029958445,Art of Cardistry - Geometry Playing Cards Vers...,USD,games,2018-03-03 01:54:57,2018-03-13 00:54:57,1000.0,US,1630.0,successful,2018
4,1036177598,Dead Duck & Zombie Chick Radio Show (vinyl pre...,USD,comics,2018-03-27 11:53:02,2018-04-26 11:53:02,7000.0,US,7656.0,successful,2018
...,...,...,...,...,...,...,...,...,...,...,...
17851,980019524,Hushme - The World's First Voice Mask For Smar...,USD,technology,2017-05-10 12:55:45,2017-06-09 12:55:45,70000.0,US,71880.0,successful,2017
17852,981977447,Get Purer Hot Water with World's Most Advanced...,USD,technology,2017-05-01 15:33:19,2017-06-15 04:00:00,125000.0,US,164866.0,successful,2017
17853,982043894,Through the Window - A photographic tale of ca...,USD,photography,2017-09-03 22:42:47,2017-10-03 22:42:47,7500.0,US,11341.0,successful,2018
17854,993614901,Send me on The Arctic Circle 2018 Arts & Scien...,USD,art,2018-01-29 16:58:19,2018-02-28 16:58:19,13500.0,US,14639.0,successful,2018


## Unit tests on filtered data

In [48]:
filtered_test_df=SparkDFDataset(filtered_df)

### 1.check if main_category in within scope

In [50]:
test_result=filtered_test_df.expect_column_values_to_be_in_set("main_category",main_categories)
print(f"categores are in scope:{'PASSED' if test_result.success else 'FAILED'}")

categores are in scope:PASSED


### 2. check if campaigns within US and in USD and are successful

In [51]:
test_result=filtered_test_df.expect_column_values_to_be_in_set("country",["US"])
print(f"campaigns done is US:{'PASSED' if test_result.success else 'FAILED'}")

campaigns done is US:PASSED


In [52]:
test_result=filtered_test_df.expect_column_values_to_be_in_set("status",["successful"])
print(f"all campaigns successful:{'PASSED' if test_result.success else 'FAILED'}")

all campaigns successful:PASSED


In [53]:
test_result=filtered_test_df.expect_column_values_to_be_in_set("currency",["USD"])
print(f"all campaigns done in USD:{'PASSED' if test_result.success else 'FAILED'}")

all campaigns done in USD:PASSED


### 3.check for uniqueness

In [54]:
test_result = filtered_test_df.expect_compound_columns_to_be_unique(["id", "assessment_year"])
print(f"""id column is unique for each assessment year: {'PASSED' if test_result.success else 'FAILED'}""")

id column is unique for each assessment year: PASSED


# standardise data

In [55]:
# reduce to 6 final metric categories and based on total pledge
metric_categories = [
    'art, crafts, photography & design',
    'dance & theater',
    'music, film & video',
    'comics, publishing & journalism',
    'games & technology',
    'food'  
]
pledge_categories = [
    '100 Thousand and under',
    'Between 100 Thousand and 500 Thousand',
    'Between 500 Thousand and 1 Million',
    'Between 1 Million and 5 Million',
    '5 Million and over'
]

### apply transformation

In [56]:
standardised_df = spark.sql(f"""
    SELECT t.*,
         CASE 
             WHEN main_category IN ('art','crafts','photography','design') THEN 'art, crafts, photography & design'
              WHEN main_category IN ('dance','theater') THEN 'dance & theater'
              WHEN main_category IN ('music','film & video') THEN 'music, film & video'
              WHEN main_category IN ('comics','publishing','journalism') THEN 'comics, publishing & journalism'
              WHEN main_category IN ('games', 'technology') THEN 'games & technology'
              WHEN main_category IN ('food') THEN 'food'                  
             END metric_category,
         CASE WHEN usd_pledged <= 100000 THEN '100 Thousand and under'
              WHEN usd_pledged > 100000 AND usd_pledged < 500000 THEN 'Between 100 Thousand and 500 Thousand'
              WHEN usd_pledged > 500000 AND usd_pledged < 1000000 THEN 'Between 500 Thousand and 1 Million'
              WHEN usd_pledged > 1000000 AND usd_pledged < 5000000 THEN 'Between 1 Million and 5 Million'
           ELSE '5 Million and over'
           END pledge_category    
    FROM filtered_campaigns t       
""")
standardised_df.createOrReplaceTempView("standardised_campaigns")
standardised_df.toPandas()

,id,name,currency,main_category,launched_at,deadline,goal_usd,country,usd_pledged,status,assessment_year,metric_category,pledge_category
0,10064783,WarmUp: A High-Protein Coffee,USD,food,2016-11-14 17:55:34,2016-12-14 17:55:34,8000.0,US,8599.0,successful,2017,food,100 Thousand and under
1,1010419089,Silver Linings by J.R. Mounts,USD,comics,2018-01-10 02:10:01,2018-02-16 05:00:00,500.0,US,2914.5,successful,2018,"comics, publishing & journalism",100 Thousand and under
2,1017132344,Princes of the Universe: a Nino Malong Art Book,USD,publishing,2016-07-31 19:03:04,2016-09-05 01:00:00,1200.0,US,1670.0,successful,2017,"comics, publishing & journalism",100 Thousand and under
3,1029958445,Art of Cardistry - Geometry Playing Cards Vers...,USD,games,2018-03-03 01:54:57,2018-03-13 00:54:57,1000.0,US,1630.0,successful,2018,games & technology,100 Thousand and under
4,1036177598,Dead Duck & Zombie Chick Radio Show (vinyl pre...,USD,comics,2018-03-27 11:53:02,2018-04-26 11:53:02,7000.0,US,7656.0,successful,2018,"comics, publishing & journalism",100 Thousand and under
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17851,980019524,Hushme - The World's First Voice Mask For Smar...,USD,technology,2017-05-10 12:55:45,2017-06-09 12:55:45,70000.0,US,71880.0,successful,2017,games & technology,100 Thousand and under
17852,981977447,Get Purer Hot Water with World's Most Advanced...,USD,technology,2017-05-01 15:33:19,2017-06-15 04:00:00,125000.0,US,164866.0,successful,2017,games & technology,Between 100 Thousand and 500 Thousand
17853,982043894,Through the Window - A photographic tale of ca...,USD,photography,2017-09-03 22:42:47,2017-10-03 22:42:47,7500.0,US,11341.0,successful,2018,"art, crafts, photography & design",100 Thousand and under
17854,993614901,Send me on The Arctic Circle 2018 Arts & Scien...,USD,art,2018-01-29 16:58:19,2018-02-28 16:58:19,13500.0,US,14639.0,successful,2018,"art, crafts, photography & design",100 Thousand and under


## Unit tests

In [57]:
std_test_df=SparkDFDataset(standardised_df)

### 1. check if campaign population equal to previous

In [60]:
filtered_total_rows = filtered_test_df.get_row_count()
test_result = std_test_df.expect_table_row_count_to_equal(filtered_total_rows)

print(f"""Total row count of standardised_df ({test_result.result['observed_value']}) \
is equal to total row count of filtered_df ({filtered_total_rows}):{'PASSED' if test_result.result['observed_value'] == filtered_total_rows else 'FAILED'}""")

Total row count of standardised_df (17856) is equal to total row count of filtered_df (17856):PASSED


# Generate Metrics

### number of successful campaigns for each metric and pledge category per assessment year

In [61]:
successful_campaigns_df = spark.sql(f"""
    SELECT assessment_year,
           metric_category,
           pledge_category,
           count(id) total_successful_campaigns
    FROM STANDARDISED_CAMPAIGNS
    GROUP BY assessment_year,
            metric_category,
            pledge_category
    ORDER BY assessment_year,
            metric_category,
            pledge_category
    """)
successful_campaigns_df.createOrReplaceTempView("s_campaigns")
successful_campaigns_df.toPandas()

,assessment_year,metric_category,pledge_category,total_successful_campaigns
0,2017,"art, crafts, photography & design",100 Thousand and under,1319
1,2017,"art, crafts, photography & design",Between 1 Million and 5 Million,2
2,2017,"art, crafts, photography & design",Between 100 Thousand and 500 Thousand,11
3,2017,"art, crafts, photography & design",Between 500 Thousand and 1 Million,3
4,2017,"comics, publishing & journalism",100 Thousand and under,1914
5,2017,"comics, publishing & journalism",Between 100 Thousand and 500 Thousand,22
6,2017,dance & theater,100 Thousand and under,468
7,2017,food,100 Thousand and under,620
8,2017,food,Between 1 Million and 5 Million,1
9,2017,food,Between 100 Thousand and 500 Thousand,6


## Unit tests

In [62]:
sc_test_df=SparkDFDataset(successful_campaigns_df)

In [63]:
test_result = sc_test_df.expect_compound_columns_to_be_unique(["assessment_year","metric_category","pledge_category"])
print(f"""metric_category column is unique for each assessment year: {'PASSED' if test_result.success else 'FAILED'}""")

metric_category column is unique for each assessment year: PASSED


In [64]:
standardised_total_rows = std_test_df.get_row_count()
test_result = sc_test_df.expect_column_sum_to_be_between('total_successful_campaigns', 
                                                                              standardised_total_rows, standardised_total_rows)
print(f"""Total sum of campaigns in metrics dataset ({test_result.result['observed_value']}) \
is equal to total rows in standardised dataset ({standardised_total_rows}): {'PASSED' if test_result.result['observed_value'] ==  standardised_total_rows else 'FAILED'}""")


Total sum of campaigns in metrics dataset (17856) is equal to total rows in standardised dataset (17856): PASSED


## Integration tests

In [65]:
raw_test_df_validation = raw_test_df.validate()
print(f"""1. Raw dataset validations: {raw_test_df_validation.success}; {raw_test_df_validation.statistics['success_percent']} successful""")
filtered_test_df_validation = filtered_test_df.validate()
print(f"""2. Filtered dataset validations: {filtered_test_df_validation.success}; {filtered_test_df_validation.statistics['success_percent']} successful""")
standardised_test_df_validation = std_test_df.validate()
print(f"""3. Standardised dataset validations: {standardised_test_df_validation.success}; {standardised_test_df_validation.statistics['success_percent']} successful""")
successful_campaigns_df_test_df_validation = sc_test_df.validate()
print(f"""4. Metrics dataset validations: {successful_campaigns_df_test_df_validation.success}; {successful_campaigns_df_test_df_validation.statistics['success_percent']} successful""")


1. Raw dataset validations: False; 47.368421052631575 successful
2. Filtered dataset validations: True; 100.0 successful
3. Standardised dataset validations: True; 100.0 successful
4. Metrics dataset validations: True; 100.0 successful


# custom unit tests

In [66]:
from great_expectations.dataset import MetaSparkDFDataset

from datetime import datetime, timedelta
from dateutil.parser import parse

In [67]:
class CustomSparkDFDataset(SparkDFDataset):
    _data_asset_type = "CustomSparkDFDataset"
    
    @MetaSparkDFDataset.column_aggregate_expectation
    def expect_column_max_to_be_less_than(
        self,
        column,
        value,
        strict=False,
        parse_strings_as_datetimes=False,
        output_strftime_format=None,
        result_format=None,
        include_config=True,
        catch_exceptions=None,
        meta=None,
    ):
        if parse_strings_as_datetimes:
            if value:
                value = parse(value)

        column_max = self.get_column_max(column, parse_strings_as_datetimes)       
        if isinstance(column_max, datetime):
            try:
                value = parse(value)
            except (ValueError, TypeError) as e:
                pass

        success = column_max < value if strict else column_max <= value
        
        if parse_strings_as_datetimes:
            if output_strftime_format:
                column_max = datetime.strftime(column_max, output_strftime_format)
            else:
                column_max = str(column_max)

        return {"success": success, "result": {"observed_value": column_max}}  
    
    @MetaSparkDFDataset.column_aggregate_expectation
    def expect_column_min_to_be_more_than(
        self,
        column,
        value,
        strict=False,
        parse_strings_as_datetimes=False,
        output_strftime_format=None,
        result_format=None,
        include_config=True,
        catch_exceptions=None,
        meta=None,
    ):
        if parse_strings_as_datetimes:
            if value:
                value = parse(value)

        column_min = self.get_column_min(column, parse_strings_as_datetimes)       
        if isinstance(column_min, datetime):
            try:
                value = parse(value)
            except (ValueError, TypeError) as e:
                pass
        
        success = column_min > value if strict else column_min >= value
        
        if parse_strings_as_datetimes:
            if output_strftime_format:
                column_min = datetime.strftime(column_min, output_strftime_format)
            else:
                column_min = str(column_min)
        return {"success": success, "result": {"observed_value": column_min}}      


In [69]:
ay_df.show()

+---------------+---------------+-------------+
|assessment_year|period_start_dt|period_end_dt|
+---------------+---------------+-------------+
|           2017|     2016-07-01|   2017-06-30|
|           2018|     2017-07-01|   2018-06-30|
+---------------+---------------+-------------+



In [71]:
ge_dataset_by_year = {}
for yr in assessment_year:
    ge_dataset_by_year[yr] = CustomSparkDFDataset(filtered_df.where(f"assessment_year = {yr}"))
ge_dataset_by_year 

{'2017': <__main__.CustomSparkDFDataset at 0x7efc4a58a2d0>,
 '2018': <__main__.CustomSparkDFDataset at 0x7efc4aced850>}

In [82]:
def get_period_end_dt(yr):
    year_filter = ay_df["assessment_year"]==yr
    end_dt = ay_df.loc[year_filter].period_end_dt.item()
    return datetime.strftime(datetime.strptime(end_dt, "%Y-%m-%d") + timedelta(days=1), "%Y-%m-%d")

In [84]:
for yr in assessment_year:
    period_end_dt = get_period_end_dt(yr)   
    test_result = ge_dataset_by_year[yr]\
                    .expect_column_max_to_be_less_than("launched_at", 
                                                       period_end_dt, 
                                                       parse_strings_as_datetimes=True, 
                                                       strict=True)
    print(f"""AY {yr} latest launched_at {test_result.result['observed_value']}""", 
          f"""< period_end_dt {period_end_dt}: {'PASSED' if test_result.success else 'FAILED'}""")


In [85]:
def get_period_start_dt(yr):
    year_filter = ay_df["assessment_year"]==yr
    return ay_df.loc[year_filter].period_start_dt.item()

In [86]:
for yr in ASSESSMENT_YEAR:
    period_start_dt = get_period_start_dt(yr)    
    test_result = ge_dataset_by_year[yr]\
                    .expect_column_min_to_be_more_than("deadline", 
                                                       period_start_dt, 
                                                       parse_strings_as_datetimes=True)
    print(f"""AY {yr} earliest deadline {test_result.result['observed_value']}""", 
          f""">= period_start_dt {period_start_dt}: {'PASSED' if test_result.success else 'FAILED'}""")   

In [ ]:
for yr, yearly_df in ge_dataset_by_year.items():
    yearly_df_validation = yearly_df.validate()
    print(f"""Filtered dataset for year {yr} validations: {yearly_df_validation.success}; {yearly_df_validation.statistics['success_percent']} successful""")
